In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from scipy import stats
from tqdm import tqdm

from assistments_workbench.config_reader import config
from assistments_workbench.models.model import classification_model, majority_model

In [2]:
data = pd.read_csv(os.path.join(config.get('localfiles', 'data_path'), 'arrs_data.csv'))

In [3]:
data['grade_diff_2'] = np.nan
data['ms_bin'] = np.nan

In [4]:
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,...,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,0,0
mean,570159.633559,33109.643174,7.479347,-0.050035,0.535543,0.992999,6.077993,16.284294,4.752159,0.707702,...,207.173676,176.095776,0.711619,0.808622,0.708009,3.559253,1.190805,0.701190,NaN,NaN
std,100392.987895,2675.788911,1.371097,0.529575,0.498747,0.083382,1.781270,26.939130,3.302097,0.062715,...,60.857489,55.756447,0.093623,0.218598,0.146376,1.350446,2.195581,0.457747,NaN,NaN
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,491757.000000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.684806,...,196.000000,130.000000,0.661031,0.666667,0.617647,3.000000,0.000000,0.000000,NaN,NaN
50%,549286.000000,31883.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.741860,3.000000,0.000000,1.000000,NaN,NaN
75%,657761.000000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,18.000000,6.000000,0.744366,...,259.000000,213.000000,0.773080,1.000000,0.811111,4.000000,2.000000,1.000000,NaN,NaN
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,NaN


In [5]:
data = data.drop_duplicates(['id', 'prereq_skill_id','prereq_easiness'], keep='first')

In [6]:
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,...,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,0,0
mean,566536.336462,33002.983077,7.483807,-0.046878,0.532241,0.992706,6.110679,16.487794,4.689068,0.709437,...,207.686588,176.569977,0.711412,0.810482,0.709818,3.540897,1.145983,0.715814,NaN,NaN
std,100241.793871,2653.785596,1.371187,0.506381,0.498972,0.085097,1.750721,26.989248,3.135947,0.061337,...,61.173924,55.723398,0.094021,0.217528,0.144406,1.314566,2.070086,0.451037,NaN,NaN
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,491142.250000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.686752,...,196.000000,130.000000,0.661031,0.681818,0.620961,3.000000,0.000000,0.000000,NaN,NaN
50%,548494.000000,31862.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.742044,3.000000,0.000000,1.000000,NaN,NaN
75%,657217.500000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,18.000000,5.000000,0.744366,...,259.000000,213.000000,0.779803,1.000000,0.811467,4.000000,1.000000,1.000000,NaN,NaN
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,NaN


In [7]:
# data = data.drop(data[(data['mastered'] ==1) & (data['ms'] < 3)].index)
# data.describe()

In [8]:
data_0 = data[data['mastered'] == 0]
data_1 = data[data['mastered'] == 1]
data_1 = data[data['ms'] >= 3]

In [9]:
group_by_col = 'ms'
grouped = data_1.groupby(group_by_col)

In [10]:
for name, group in grouped:
    if name >=3 and name <=4:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 1
    elif name >=5 and name <=6:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 2
    else:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 3

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
data = pd.concat([data_0, data_1])
data.loc[(data['mastered'] == 0),'ms_bin'] = 0
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,...,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,0,20361.000000
mean,565565.921910,32998.107411,7.477236,-0.047787,0.530720,0.987427,6.142773,16.587790,4.754285,0.709003,...,207.064093,176.253475,0.711408,0.809368,0.712406,3.572811,1.177251,0.717941,NaN,1.486567
std,100134.672163,2652.693206,1.375117,0.507048,0.499068,0.111425,1.706971,27.053813,3.136324,0.061474,...,61.195012,55.849685,0.094005,0.218317,0.142913,1.298108,2.094446,0.450013,NaN,0.772774
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,491085.000000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.684806,...,196.000000,130.000000,0.661031,0.666667,0.625407,3.000000,0.000000,0.000000,NaN,1.000000
50%,548359.000000,31862.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.742857,3.000000,0.000000,1.000000,NaN,1.000000
75%,657189.000000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,19.000000,6.000000,0.744366,...,259.000000,213.000000,0.779803,1.000000,0.812536,4.000000,2.000000,1.000000,NaN,2.000000
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,3.000000


In [12]:
group_by_col = 'grade_diff'
grouped = data.groupby(group_by_col)

In [13]:
for name, group in grouped:
    if name == 0:
        # on grade
        data.loc[(data[group_by_col] == name),'grade_diff_2'] = 0
    else:
        if name > 0:
            # above grade
             data.loc[(data[group_by_col] == name),'grade_diff_2'] = 1
        elif name == -1:
            # one year ago
            data.loc[(data[group_by_col] == name),'grade_diff_2'] = 2
        else:
            # longer ago
            data.loc[(data[group_by_col] == name),'grade_diff_2'] = 3

In [14]:
data.describe()
data.to_csv('cleaned.csv')

In [15]:
pfa_data = data[['skill_id', 'easiness', 'right', 'wrong', 'correct']]

In [16]:
pfa_data.describe()

,skill_id,easiness,right,wrong,correct
count,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000
mean,207.064093,0.712406,3.572811,1.177251,0.717941
std,61.195012,0.142913,1.298108,2.094446,0.450013
min,46.000000,0.000000,0.000000,0.000000,0.000000
25%,196.000000,0.625407,3.000000,0.000000,0.000000
50%,217.000000,0.742857,3.000000,0.000000,1.000000
75%,259.000000,0.812536,4.000000,2.000000,1.000000
max,296.000000,1.000000,25.000000,54.000000,1.000000


In [17]:
# data = data[data['adaptive_mode'] == 0]

In [18]:
label = 'correct'
all_features = pfa_data.columns.tolist()
all_features.remove(label)
predictors = all_features
categorical_features = ['skill_id']
k = 5

In [19]:
model = LogisticRegression()
print classification_model(model, pfa_data, predictors, label, categorical_features, k, False)

(0.72952151900399043, 0.13455336582358093, 0.41867174147313796, 0.74151609115952144)


In [20]:
label = 'correct'
all_features = data.columns.tolist()
all_features.remove(label)
all_features.remove('id')
all_features.remove('grade_diff')
all_features.remove('ms')
all_features.remove('right')
all_features.remove('wrong')
all_features.remove('mastered')
# all_features.remove('delay_days')
predictors = all_features
categorical_features = ['student_class_id', 'adaptive_mode', 'skill_id', 'prereq_skill_id', 'grade_diff_2', 'ms_bin']

In [21]:
all_feature_set = set(all_features)
categorical_feature_set = set(categorical_features)
continue_features = all_feature_set.difference(categorical_feature_set)
continue_data = data[list(continue_features)]
continue_data.corr()

,class_assignment_performance,class_performance,class_grade,work_delay_in_day,prereq_easiness,delay_days,easiness,prereq_performance
class_assignment_performance,1.000000,0.393285,-0.256656,0.052245,0.055615,0.253018,0.499692,0.070750
class_performance,0.393285,1.000000,-0.146811,-0.079493,-0.111154,0.089616,0.039204,0.123106
class_grade,-0.256656,-0.146811,1.000000,-0.243250,-0.260933,-0.130576,-0.287890,-0.055066
work_delay_in_day,0.052245,-0.079493,-0.243250,1.000000,0.147270,0.123557,0.087420,0.004675
prereq_easiness,0.055615,-0.111154,-0.260933,0.147270,1.000000,0.028987,0.142034,0.379050
delay_days,0.253018,0.089616,-0.130576,0.123557,0.028987,1.000000,0.189770,0.045144
easiness,0.499692,0.039204,-0.287890,0.087420,0.142034,0.189770,1.000000,0.030780
prereq_performance,0.070750,0.123106,-0.055066,0.004675,0.379050,0.045144,0.030780,1.000000


In [22]:
# model = ensemble.GradientBoostingClassifier()
model = LogisticRegression()

In [23]:
print majority_model(data, label)

(0.5, nan, 0.53109211816271495, 0.71794116202544078)


In [24]:
print classification_model(model, data, predictors, label, categorical_features, k, True)

(0.77466447961911022, 0.20034021726617976, 0.40240912096217257, 0.76504101430557914)


In [25]:
single_feature_performance = pd.DataFrame(index = range(len(all_features)), columns = ['feature', 'AUC', 'R2', 'r'])

In [26]:
for n, feature in enumerate(all_features):
    if feature in categorical_features:
        cf = [feature]
        r = np.nan
    else:
        cf = None
        r, _ = stats.pearsonr(data[feature], data[label])
    
    a,b,c,d = classification_model(model, data, [feature], label, cf, k)
    
    single_feature_performance.ix[n,'feature'] = feature
    single_feature_performance.ix[n,'AUC'] = a
    single_feature_performance.ix[n,'R2'] = b
    single_feature_performance.ix[n,'r'] = r
    

In [27]:
single_feature_performance = single_feature_performance.sort_values(by='AUC', axis=0, ascending=False).reset_index(drop=True)

single_feature_performance

,feature,AUC,R2,r
0,class_assignment_performance,0.725477,0.131392,0.364248
1,easiness,0.712814,0.116892,0.344288
2,student_class_id,0.631542,0.0452593,NaN
3,ms_bin,0.596727,0.0333462,NaN
4,prereq_skill_id,0.587446,0.0228727,NaN
5,skill_id,0.584551,0.0233238,NaN
6,class_performance,0.580041,0.0160781,0.126669
7,delay_days,0.570672,0.0145348,0.122441
8,prereq_performance,0.554921,0.00826084,0.0896779
9,class_grade,0.549633,0.0103878,-0.103835


In [28]:
num_features = 5
num_combinations = 500

In [29]:
feature_column_names = ['feature'+str(x+1) for x in range(num_features)]
feature_combinations_performance = pd.DataFrame(index=range(num_combinations), columns = feature_column_names + ['AUC', 'R2'])

for combination in tqdm(range(num_combinations)):
    random_features = sorted(np.random.choice(len(all_features), num_features, replace=False))
    
    # store the feature names
    random_feature_names = [all_features[x] for x in random_features]
    for i in range(len(random_features)):
        feature_combinations_performance.ix[combination, feature_column_names[i]] = random_feature_names[i]
    
    cf = []
    for f in random_feature_names:
        if f in categorical_features:
            cf.append(f)
    if len(cf) == 0:
        cf = None
        
    a,b,c,d = classification_model(model, data, random_feature_names, label, cf, k)
    feature_combinations_performance.ix[combination,'AUC'] = a
    feature_combinations_performance.ix[combination,'R2'] = b


In [30]:
feature_combinations_performance = feature_combinations_performance.sort_values(by='AUC', axis=0, ascending=False).reset_index(drop=True)
feature_combinations_performance.ix[:10,:]

,feature1,feature2,feature3,feature4,feature5,AUC,R2
0,class_assignment_performance,prereq_skill_id,prereq_performance,easiness,ms_bin,0.759844,0.177861
1,work_delay_in_day,class_assignment_performance,prereq_easiness,prereq_performance,easiness,0.759381,0.175246
2,student_class_id,work_delay_in_day,class_assignment_performance,easiness,ms_bin,0.759346,0.17864
3,delay_days,work_delay_in_day,class_assignment_performance,prereq_performance,easiness,0.759213,0.174792
4,student_class_id,work_delay_in_day,class_assignment_performance,skill_id,easiness,0.758962,0.178885
5,adaptive_mode,class_assignment_performance,skill_id,prereq_performance,easiness,0.758889,0.178336
6,student_class_id,delay_days,class_assignment_performance,prereq_performance,easiness,0.75887,0.178418
7,delay_days,class_assignment_performance,prereq_skill_id,prereq_performance,easiness,0.758509,0.176092
8,delay_days,class_assignment_performance,prereq_easiness,prereq_performance,easiness,0.758346,0.173753
9,adaptive_mode,class_assignment_performance,prereq_performance,easiness,ms_bin,0.758205,0.174474
